In [90]:
import importlib
import numpy as np
import NN
importlib.reload(NN)

<module 'NN' from 'd:\\Work\\Programming\\Jupyter\\Customer Segmentation NN\\NN.py'>

In [91]:
layers = [
    NN.DenseLayer(2, 0, NN.dummy_activation, NN.dummy_activation, True, False),
    NN.DenseLayer(4, 2, NN.sigmoid, NN.sigmoid_prime),
    NN.DenseLayer(2, 4, NN.sigmoid, NN.sigmoid_prime, False, True)
]
X = np.array([[1, 1], [1, 2]])
y = np.array([[0], [1]])


In [92]:
a2 = layers[1].forward_propagation(X)
print(layers[2].loss(a2, y, 2))
for i in range(100):
    NN.backward_propagate(layers, X, y, 0.01)
    
a2 = layers[1].forward_propagation(X)
print(layers[2].loss(a2, y, 2))

[[-0.25577024  0.40866603 -0.15528593 -0.51242692 -0.33958153]
 [ 0.27860503  0.03935532  2.1142968  -0.16885269 -0.15172112]]
[[-0.25577024  2.77350642 -4.44738097 -2.5695866   2.58854144]
 [ 0.27860503 -2.48190474  5.45855576  1.3048295  -2.91411167]]


array([[ 0],
       [-1]])